In [1]:
%load_ext watermark
%watermark

Last updated: 2024-04-18T12:29:51.555848-05:00

Python implementation: CPython
Python version       : 3.10.11
IPython version      : 8.13.2

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing, feature_extraction
from sklearn import preprocessing, feature_extraction
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import KNeighborsClassifier
#import category_encoders
from sklearn.model_selection import cross_validate
from category_encoders import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics

In [3]:
consumidores = pd.read_csv("C:/Users/Danny/Desktop/Analisis tesis/4 - Pipelines/data/df_consumo_varObjetivo.csv")

In [4]:
consumidores.head()

,departamento,residentes_hogar,d2_04_num_hijos,per_edad_tipo,estrato_tipo,situacion_tipo,frecuencia_consumo_marihuana_tipo,frecuencia_consumo_cocaina_tipo,frecuencia_consumo_basuco_tipo,per_sexo_tipo,...,g_06_a_posibilidad_conseguir_marihuana_tipo,g_06_b_posibilidad_conseguir_cocaina_tipo,g_06_c_posibilidad_conseguir_basuco_tipo,g_07_alguien_ofrecio_comprar_probar_sp_tipo,g_01_a_num_familiares_consumen_sp_imp_tipo,g_02_a_num_amigos_consumen_sp_imp_tipo,g_08_a_ofrecieron_marihuana_imp_tipo,g_08_b_ofrecieron_cocaina_imp_tipo,g_08_c_ofrecieron_basuco_imp_tipo,CatRiesgo
0,Guaviare,5,0,Joven,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
1,Cordoba,4,3,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
2,Caqueta,4,2,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
3,Bolivar,6,0,Adulto,Estrato alto,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_No_podria_conseguir,g_06_b_No_podria_conseguir,g_06_c_No_podria_conseguir,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
4,Tolima,4,1,Joven,Estrato bajo,Buscando trabajo,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,1


Como vimos en un apartado anterior, en este modelo vamos a modificar cada variable en función de su tipo. Al conjunto de pasos que siguen los datos se le llama comúnmente **Pipelines** (literalmente, sistemas de tuberias).

**INCISO**

vamos a modificar dos transformadores de scikitlearn para que sean compatibles con pipelines. Este paso es necesario en la version actual de scikit-learn, pero seguramente será arreglado en el futuro

In [5]:
class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, columns, output_type="matrix"):
        self.columns = columns
        self.output_type = output_type

    def transform(self, X, **transform_params):
        if isinstance(X, list):
            X = pd.DataFrame.from_dict(X)
        if self.output_type == "matrix":
            return X[self.columns].to_numpy()
        elif self.output_type == "dataframe":
            return X[self.columns]
        raise Exception("output_type tiene que ser matrix o dataframe")
        
    def fit(self, X, y=None, **fit_params):
        return self

In [6]:
variable_objetivo = ["CatRiesgo"]
variables_independientes = consumidores.drop(variable_objetivo, axis=1).columns

In [7]:
datos_numericos = consumidores[variables_independientes].select_dtypes([int, float])
col_numericas = datos_numericos.columns
col_no_numericas = consumidores[variables_independientes].select_dtypes([object]).columns

dict_var_ordinales = {
    "per_edad_tipo": ["Adolescente", "Joven", "Adulto", "Persona mayor"],
    "estrato_tipo":["Estrato bajo", "Estrato medio", "Estrato alto"],
    "d_08_estado_salud_tipo":["d_08_Muy_malo","d_08_Malo","d_08_Regular","d_08_Bueno","d_08_Muy_bueno"],
    "frecuencia_consumo_marihuana_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_cocaina_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_basuco_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = consumidores[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = consumidores[col_categoricas]

Un pipeline de sklearn se define como una secuencia de pasos. Cada paso se define con una tupla de forma `[nombre del paso, transformador]`

Por ejemplo, si queremos crear un pipeline que procese las variables numéricas, primero imputándolas y después estandarizandolas, podriamos crear un pipeline

Creamos ahora los pipelines para cada tipo de variable. Es necesario aclarar que se va ha hacer uso de la libreria Category Encoders par ael procesamiento de variables categoricas y categoricas ordinales

In [8]:
pipeline_numerico = Pipeline([
    ["selector_numerico", ColumnExtractor(columns=col_numericas)],
    ["transformador_numerico", Pipeline([
        ("imputador_numerico", SimpleImputer(missing_values=np.nan, strategy='mean')),
        ("escalador_numerico", preprocessing.MinMaxScaler())
    ])]
])

pipeline_numerico.fit_transform(consumidores)[:5]

array([[0.30769231, 0.        ],
       [0.23076923, 0.25      ],
       [0.23076923, 0.16666667],
       [0.38461538, 0.        ],
       [0.23076923, 0.08333333]])

Ahora se procede a crear el pipeline categorico

In [9]:
pipeline_categorico = Pipeline([
    ("selector_categorica", ColumnExtractor(
        columns=col_categoricas,
        output_type="dataframe")),
    ("transformador_categorico", OneHotEncoder(
        cols=col_categoricas, handle_unknown="impute")),
])

pipeline_categorico.fit_transform(consumidores)[:5]

,g_08_a_ofrecieron_marihuana_imp_tipo_1,g_08_a_ofrecieron_marihuana_imp_tipo_2,g_08_a_ofrecieron_marihuana_imp_tipo_3,g_08_a_ofrecieron_marihuana_imp_tipo_4,g_08_a_ofrecieron_marihuana_imp_tipo_5,d_10_poco_interes_tipo_1,d_10_poco_interes_tipo_2,g_02_amigos_consumen_sp_tipo_1,g_02_amigos_consumen_sp_tipo_2,g_06_a_posibilidad_conseguir_marihuana_tipo_1,...,departamento_23,departamento_24,departamento_25,departamento_26,departamento_27,departamento_28,departamento_29,departamento_30,departamento_31,departamento_32
0,1,0,0,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


`category_encoders.OrdinalEncoders` requiere un mapeado específico para cada columna, indicando el número por el que queremos reemplazar cada nivel de cada variable ordinal. Si no especificamos un mapa, asume que el orden es aleatorio.

In [10]:
niveles_ordinales = []
for col, levels in dict_var_ordinales.items():
    niveles_ordinales.append({"col": col, 
                              "mapping": dict(zip(levels, range(len(levels))))})

In [11]:
pipeline_ordinal = Pipeline([
    ("selector_ordinal", ColumnExtractor(columns=col_ordinales, output_type="dataframe")),
    ("transformador_ordinal", OrdinalEncoder(mapping=niveles_ordinales))
])
pipeline_ordinal.fit_transform(consumidores)[:5]

,per_edad_tipo,estrato_tipo,d_08_estado_salud_tipo,frecuencia_consumo_marihuana_tipo,frecuencia_consumo_cocaina_tipo,frecuencia_consumo_basuco_tipo
0,1,0,1,0,0,0
1,2,0,2,0,0,0
2,2,0,3,0,0,0
3,2,2,3,0,0,0
4,1,0,3,0,0,0


Ya tenemos una manera de, dado un conjunto de datos, separarlos y aplicar distintas transformaciones a cada variable. Nos falta una manera de, una vez se han transformado, reunirlas de nuevo.

Para ello podemos usar `FeatureUnion`, que simplemente toma un conjunto de pasos de un pipeline y los une.

In [12]:
pipeline_procesado = FeatureUnion([
    ('variables_numericas', pipeline_numerico),
    ('variables_ordinales', pipeline_ordinal),
    ('variables_categoricas', pipeline_categorico),
])

In [13]:
pipeline_procesado

FeatureUnion(transformer_list=[('variables_numericas',
                                Pipeline(steps=[('selector_numerico',
                                                 ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                ['transformador_numerico',
                                                 Pipeline(steps=[('imputador_numerico',
                                                                  SimpleImputer()),
                                                                 ('escalador_numerico',
                                                                  MinMaxScaler())])]])),
                               ('variables_ordinales',
                                Pipeline(steps=[('select...
                                                                     'd_01_aporta_dinero_hogar_tipo',
                                                                     'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                     'g_04_probaria_sp_tipo',
                                                                     'vivienda_tipo',
                                                                     'g_01_familiares_consumen_sp_tipo',
                                                                     'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                     'd2_05_nivel_educativo_tipo',
                                                                     'per_sexo_tipo',
                                                                     'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                     'g_05_posibilidad_probar_sp_tipo', ...],
                                                               handle_unknown='impute'))]))])

In [14]:
consumidores_procesado = pipeline_procesado.fit_transform(consumidores)
consumidores_procesado.shape

(10000, 145)

Finalmente, necesitamos añadir un estimador al final para predecir en base a los datos transformados, pero primero se separa los datos en datos de entrenamiento y datos de test

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
     consumidores[variables_independientes], consumidores["CatRiesgo"],
     test_size=0.20, random_state=42
)

In [16]:
k_categorias = len(y_train.unique())
k_categorias

5

In [17]:
estimador = KNeighborsClassifier(n_neighbors=5)

Se debe realizar la separacion de datos en datos de entrenamiento y datos de test y ademas se debe establecer el estimador a usar en el modelo

In [18]:
pipeline_estimador = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', estimador)
])

Para el fit se pasan los datos que se obtienen de entrenamiento y para el predict los datos de test

In [19]:
pipeline_estimador.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                      'g_04_probaria_sp_tipo',
                                                                                      'vivienda_tipo',
                                                                                      'g_01_familiares_consumen_sp_tipo',
                                                                                      'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                      'd2_05_nivel_educativo_tipo',
                                                                                      'per_sexo_tipo',
                                                                                      'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                      'g_05_posibilidad_probar_sp_tipo', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador', KNeighborsClassifier())])

In [20]:
predicciones = pipeline_estimador.predict(X_test)

In [21]:
predicciones

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

Se evalua el modelo segun lo obtenido

El beneficio de los pipelines, no solo es tener codigo mas legible y poder gestionar de forma ordenada todo el ciclo de vida del modelado, sino que los pipelines tienen todos los beneficios de los objetos de scikitlearn, por ejemplo, podemos usar validacion cruzada directamente con el pipeline.

In [22]:
resultados = {}

def exactitud_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.accuracy_score(y, preds)

def precision_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.precision_score(y, preds, average="macro")

def exhaustividad_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.recall_score(y, preds, average="macro")

def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.f1_score(y, preds, average="macro")


In [23]:
def evaluar_modelo(prmPipeline,X, y, semillas):
    restultados_estimador = {}
    restultados_estimador["exactitud"] = cross_validate(prmPipeline, X, y,
                                           scoring=exactitud_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["precision"] = cross_validate(prmPipeline, X, y,
                                           scoring=precision_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["exhaustividad"] = cross_validate(prmPipeline, X, y,
                                           scoring=exhaustividad_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["f1_score"] = cross_validate(prmPipeline, X, y,
                                           scoring=f1_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    return restultados_estimador

In [24]:
#Funcion para poder visuallizar los resultados de las metricas
def ver_resultados():
    df_res = pd.DataFrame()
    for modelo, metricas in resultados.items():
        for metrica, valores in metricas.items():
            for medida, arreglo in valores.items():
                # Calcula la media de cada array y agrega una nueva fila al DataFrame
                df_res.loc[f"{modelo}_{metrica}", medida] = np.mean(arreglo)
                
    return df_res

In [25]:
estimador_agregado = pipeline_estimador.named_steps['estimador']
resultados["knn"] = evaluar_modelo(estimador_agregado, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
knn_exactitud,0.030640,0.093602,0.909500,0.942473
knn_precision,0.027039,0.061876,0.893786,0.931175
knn_exhaustividad,0.029359,0.063813,0.872923,0.921687
knn_f1_score,0.029958,0.063433,0.877132,0.924875


### Optimizacion de hiperparametros

In [26]:
knn_param = {
    'n_neighbors': [1, 3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
    'algorithm': ['kd_tree','ball_tree','brute'],
    'leaf_size': [10, 20, 30, 40],
    'metric': ['euclidean', 'manhattan']
}

In [27]:
buscador_knn = GridSearchCV(estimador, knn_param, cv=50, scoring='accuracy', n_jobs=-1)

In [28]:
pipeline_estimador_op = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', buscador_knn)
])

In [29]:
pipeline_estimador_op.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_05_posibilidad_probar_sp_tipo', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 GridSearchCV(cv=50, estimator=KNeighborsClassifier(),
                              n_jobs=-1,
                              param_grid={'algorithm': ['kd_tree', 'ball_tree',
                                                        'brute'],
                                          'leaf_size': [10, 20, 30, 40],
                                          'metric': ['euclidean', 'manhattan'],
                                          'n_neighbors': [1, 3, 5, 7, 9],
                                          'p': [1, 2],
                                          'weights': ['uniform', 'distance']},
                              scoring='accuracy'))])

In [30]:
pipeline_estimador_op.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

Se crea un pipeline con el mejor estimador obtenido

In [31]:
print(buscador_knn.best_score_)
print(buscador_knn.best_estimator_)

0.9071250000000001
KNeighborsClassifier(algorithm='kd_tree', leaf_size=20, metric='euclidean',
                     n_neighbors=9, p=1, weights='distance')


In [32]:
pipeline_estimador_op_best = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', buscador_knn.best_estimator_)
])

In [33]:
pipeline_estimador_op_best.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_01_familiares_consumen_sp_tipo',
                                                                                      'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                      'd2_05_nivel_educativo_tipo',
                                                                                      'per_sexo_tipo',
                                                                                      'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                      'g_05_posibilidad_probar_sp_tipo', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 KNeighborsClassifier(algorithm='kd_tree', leaf_size=20,
                                      metric='euclidean', n_neighbors=9, p=1,
                                      weights='distance'))])

In [34]:
pipeline_estimador_op_best.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [35]:
estimador_best = pipeline_estimador_op_best.named_steps['estimador']
resultados["knn_best"] = evaluar_modelo(estimador_best, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
knn_exactitud,0.030640,0.093602,0.909500,0.942473
knn_precision,0.027039,0.061876,0.893786,0.931175
knn_exhaustividad,0.029359,0.063813,0.872923,0.921687
knn_f1_score,0.029958,0.063433,0.877132,0.924875
knn_best_exactitud,1.006713,1.780749,0.912100,1.000000
knn_best_precision,1.008348,1.785483,0.897955,1.000000
knn_best_exhaustividad,0.989680,1.790102,0.879391,1.000000
knn_best_f1_score,1.013770,1.816675,0.882520,1.000000


### Segunda optimizacion

In [36]:
knn_param2 = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 15],
    'weights': ['uniform'],
    'p': [1],
    'algorithm': ['kd_tree'],
    'leaf_size': [10, 20, 30, 40, 50],
    'metric': ['euclidean']
}

In [37]:
buscador_knn2 = GridSearchCV(estimador, knn_param2, cv=50, scoring='accuracy', n_jobs=-1)

In [38]:
pipeline_estimador_op2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', buscador_knn2)
])

In [39]:
pipeline_estimador_op2.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                      'g_05_posibilidad_probar_sp_tipo', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 GridSearchCV(cv=50, estimator=KNeighborsClassifier(),
                              n_jobs=-1,
                              param_grid={'algorithm': ['kd_tree'],
                                          'leaf_size': [10, 20, 30, 40, 50],
                                          'metric': ['euclidean'],
                                          'n_neighbors': [1, 3, 5, 7, 9, 11,
                                                          15],
                                          'p': [1], 'weights': ['uniform']},
                              scoring='accuracy'))])

In [40]:
pipeline_estimador_op2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [41]:
print(buscador_knn2.best_score_)
print(buscador_knn2.best_estimator_)

0.907
KNeighborsClassifier(algorithm='kd_tree', leaf_size=10, metric='euclidean',
                     n_neighbors=11, p=1)


In [42]:
pipeline_estimador_op_best2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', buscador_knn2.best_estimator_)
])

In [43]:
pipeline_estimador_op_best2.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_01_familiares_consumen_sp_tipo',
                                                                                      'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                      'd2_05_nivel_educativo_tipo',
                                                                                      'per_sexo_tipo',
                                                                                      'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                      'g_05_posibilidad_probar_sp_tipo', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 KNeighborsClassifier(algorithm='kd_tree', leaf_size=10,
                                      metric='euclidean', n_neighbors=11,
                                      p=1))])

In [44]:
pipeline_estimador_op_best2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [45]:
estimador_best_2 = pipeline_estimador_op_best2.named_steps['estimador']
resultados["knn_best_2"] = evaluar_modelo(estimador_best_2, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
knn_exactitud,0.030640,0.093602,0.909500,0.942473
knn_precision,0.027039,0.061876,0.893786,0.931175
knn_exhaustividad,0.029359,0.063813,0.872923,0.921687
knn_f1_score,0.029958,0.063433,0.877132,0.924875
knn_best_exactitud,1.006713,1.780749,0.912100,1.000000
knn_best_precision,1.008348,1.785483,0.897955,1.000000
knn_best_exhaustividad,0.989680,1.790102,0.879391,1.000000
knn_best_f1_score,1.013770,1.816675,0.882520,1.000000
knn_best_2_exactitud,1.085561,1.893832,0.913200,0.930871
knn_best_2_precision,1.054664,1.868789,0.900797,0.920326


### Tercera optimizacion

In [46]:
knn_param3 = {
    'n_neighbors': [9, 11, 13, 15, 17],
    'weights': ['distance'],
    'p': [1],
    'algorithm': ['kd_tree'],
    'leaf_size': [10, 20, 30, 40, 50],
    'metric': ['euclidean']
}

In [47]:
buscador_knn3 = GridSearchCV(estimador, knn_param3, cv=50, scoring='accuracy', n_jobs=-1)

In [48]:
pipeline_estimador_op3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', buscador_knn3)
])

In [49]:
pipeline_estimador_op3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                      'g_05_posibilidad_probar_sp_tipo', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 GridSearchCV(cv=50, estimator=KNeighborsClassifier(),
                              n_jobs=-1,
                              param_grid={'algorithm': ['kd_tree'],
                                          'leaf_size': [10, 20, 30, 40, 50],
                                          'metric': ['euclidean'],
                                          'n_neighbors': [9, 11, 13, 15, 17],
                                          'p': [1], 'weights': ['distance']},
                              scoring='accuracy'))])

In [50]:
pipeline_estimador_op3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [51]:
print(buscador_knn3.best_score_)
print(buscador_knn3.best_estimator_)

0.907625
KNeighborsClassifier(algorithm='kd_tree', leaf_size=10, metric='euclidean',
                     n_neighbors=11, p=1, weights='distance')


In [52]:
pipeline_estimador_op_best3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', buscador_knn3.best_estimator_)
])

In [53]:
pipeline_estimador_op_best3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_01_familiares_consumen_sp_tipo',
                                                                                      'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                      'd2_05_nivel_educativo_tipo',
                                                                                      'per_sexo_tipo',
                                                                                      'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                      'g_05_posibilidad_probar_sp_tipo', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 KNeighborsClassifier(algorithm='kd_tree', leaf_size=10,
                                      metric='euclidean', n_neighbors=11, p=1,
                                      weights='distance'))])

In [54]:
pipeline_estimador_op_best3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [55]:
estimador_best_3= pipeline_estimador_op_best3.named_steps['estimador']
resultados["knn_best_3"] = evaluar_modelo(estimador_best_3, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
knn_exactitud,0.030640,0.093602,0.909500,0.942473
knn_precision,0.027039,0.061876,0.893786,0.931175
knn_exhaustividad,0.029359,0.063813,0.872923,0.921687
knn_f1_score,0.029958,0.063433,0.877132,0.924875
knn_best_exactitud,1.006713,1.780749,0.912100,1.000000
knn_best_precision,1.008348,1.785483,0.897955,1.000000
knn_best_exhaustividad,0.989680,1.790102,0.879391,1.000000
knn_best_f1_score,1.013770,1.816675,0.882520,1.000000
knn_best_2_exactitud,1.085561,1.893832,0.913200,0.930871
knn_best_2_precision,1.054664,1.868789,0.900797,0.920326


### Cuarta optimizacion

In [56]:
knn_param4 = {
    'n_neighbors': [13, 15, 17],
    'weights': ['distance'],
    'p': [1],
    'algorithm': ['kd_tree'],
    'leaf_size': [10, 15],
    'metric': ['euclidean']
}

In [57]:
buscador_knn4 = GridSearchCV(estimador, knn_param4, cv=50, scoring='accuracy', n_jobs=-1)

In [58]:
pipeline_estimador_op4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', buscador_knn4)
])

In [59]:
pipeline_estimador_op4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                      'g_05_posibilidad_probar_sp_tipo', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 GridSearchCV(cv=50, estimator=KNeighborsClassifier(),
                              n_jobs=-1,
                              param_grid={'algorithm': ['kd_tree'],
                                          'leaf_size': [10, 15],
                                          'metric': ['euclidean'],
                                          'n_neighbors': [13, 15, 17], 'p': [1],
                                          'weights': ['distance']},
                              scoring='accuracy'))])

In [60]:
print(buscador_knn4.best_score_)
print(buscador_knn4.best_estimator_)

0.9051250000000001
KNeighborsClassifier(algorithm='kd_tree', leaf_size=10, metric='euclidean',
                     n_neighbors=17, p=1, weights='distance')


In [61]:
pipeline_estimador_op4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [62]:
pipeline_estimador_op_best4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', buscador_knn4.best_estimator_)
])

In [63]:
pipeline_estimador_op_best4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 Pipeline(steps=[('selector_numerico',
                                                                  ColumnExtractor(columns=Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))),
                                                                 ['transformador_numerico',
                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                   SimpleImputer()),
                                                                                  ('escalador_numerico',
                                                                                   MinMaxScaler())])]])),
                                                ('variables_ordin...
                                                                                      'g_01_familiares_consumen_sp_tipo',
                                                                                      'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                      'd2_05_nivel_educativo_tipo',
                                                                                      'per_sexo_tipo',
                                                                                      'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                      'g_05_posibilidad_probar_sp_tipo', ...],
                                                                                handle_unknown='impute'))]))])),
                ('estimador',
                 KNeighborsClassifier(algorithm='kd_tree', leaf_size=10,
                                      metric='euclidean', n_neighbors=17, p=1,
                                      weights='distance'))])

In [64]:
pipeline_estimador_op_best4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [65]:
estimador_best_4 = pipeline_estimador_op_best4.named_steps['estimador']
resultados["knn_best_4"] = evaluar_modelo(estimador_best_4, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
knn_exactitud,0.030640,0.093602,0.909500,0.942473
knn_precision,0.027039,0.061876,0.893786,0.931175
knn_exhaustividad,0.029359,0.063813,0.872923,0.921687
knn_f1_score,0.029958,0.063433,0.877132,0.924875
knn_best_exactitud,1.006713,1.780749,0.912100,1.000000
knn_best_precision,1.008348,1.785483,0.897955,1.000000
knn_best_exhaustividad,0.989680,1.790102,0.879391,1.000000
knn_best_f1_score,1.013770,1.816675,0.882520,1.000000
knn_best_2_exactitud,1.085561,1.893832,0.913200,0.930871
knn_best_2_precision,1.054664,1.868789,0.900797,0.920326


### Exportar con joblib el mejor pipeline obtenido

Tenemos que exportar 3 cosas para poder recrear el Pipeline:

**1. El pipeline entrenado.**

Podemos usar la libreria de scikit-learn `joblib` para guardar nuestro pipeline al disco duro. Joblib es una libreria basada en pickle que sirve para guardar objetos de python al disco duro, pero que está optimizado para scikit-learn.

In [66]:
import joblib

In [67]:
joblib.dump(pipeline_estimador_op_best3, 'pipeline_knn_best.pkl')

['pipeline_knn_best.pkl']

In [68]:
clf = joblib.load('C:/Users/Danny/Desktop/Analisis_Modelo/Alisis Danny/4 - Pipelines/Modelo KNN/pipeline_knn_best.pkl')

In [69]:
clf.predict(consumidores.head())

array([0, 0, 0, 0, 1], dtype=int64)